# Model WL Profiles (different redshift inputs)
## Model profiles using different type of source redshift information as input

In this example we model lensing profiles by giving as input either : 
- discrete source redshifts, 
- a redshift distribution function,
- the value of the mean beta parameters : 
$\langle \beta_s \rangle = \left\langle \frac{D_{LS}}{D_S}\frac{D_\infty}{D_{L,\infty}}\right\rangle$ ,
$\langle \beta_s^2 \rangle = \left\langle \left(\frac{D_{LS}}{D_S}\frac{D_\infty}{D_{L,\infty}}\right)^2 \right\rangle$

In [ ]:
import os

## Uncomment the following line if you want to use a specific modeling backend among 'ct' (cluster-toolkit), 'ccl' (CCL) or 'nc' (Numcosmo). Default is 'ccl'
# os.environ['CLMM_MODELING_BACKEND'] = 'nc'

In [ ]:
import clmm
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Math, display

%matplotlib inline

In [ ]:
from paper_formating import add_grid, prep_plot

Make sure we know which version we're using

In [ ]:
clmm.__version__

## Import mock data module and setup the configuration 

In [ ]:
from clmm import Cosmology
from clmm.redshift.distributions import *
from clmm.support import mock_data as mock

Mock data generation requires a defined cosmology

In [ ]:
mock_cosmo = Cosmology(H0=70.0, Omega_dm0=0.27 - 0.045, Omega_b0=0.045, Omega_k0=0.0)

Mock data generation requires some cluster information

In [ ]:
cosmo = mock_cosmo

# cluster properties from https://arxiv.org/pdf/1611.03866.pdf
cluster_id = "SPT-CL J0000−5748"
cluster_m = 4.56e14  # M500,c
cluster_z = 0.702
cluster_ra = 0.2499
cluster_dec = -57.8064
concentration = 5  # (arbitrary value, not from the paper)

# source redshift distribution properties
cluster_beta_s_mean = 0.466
cluster_beta_s2_mean = 0.243
ngal_density = (
    26.0 * 100
)  # density of source galaxies per arcmin^2 # (arbitrary value, not from the paper)
model_z_distrib_dict = {"func": desc_srd, "name": "desc_srd"}
delta_z_cut = 0.1
zsrc_min = cluster_z + delta_z_cut
zsrc_max = 3.0

## Different inputs for the source redshifts

### Discrete redshifts

#### Generate the mock source catalog

The CLMM mock data generation will provide, among other things, a redshift value for each background galaxy that is draw from the redshift distribution given by `model_z_distrib_dict`.

In [ ]:
np.random.seed(0)
source_catalog = mock.generate_galaxy_catalog(
    cluster_m,
    cluster_z,
    concentration,
    cosmo,
    model_z_distrib_dict["name"],
    delta_so=500,
    massdef="critical",
    zsrc_min=zsrc_min,
    zsrc_max=zsrc_max,
    ngal_density=ngal_density,
    cluster_ra=cluster_ra,
    cluster_dec=cluster_dec,
)

### Beta parameters
From this udnerlying redshift distribution, one may directly compute the average $\langle\beta_s\rangle$ and $\langle\beta_s^2\rangle$ quantities

In [ ]:
beta_label = lambda beta: rf"\langle\beta_s\rangle = {beta:.2f}"
beta_sq_label = lambda beta_sq: rf"\langle\beta_s^2\rangle = {beta_sq:.2f}"

It is also possible to compute $\langle\beta_s\rangle$ and $\langle\beta_s^2\rangle$ using galaxy shape weights:

### Visualisation

In [ ]:
z = np.linspace(0, zsrc_max, 1000)
fig = prep_plot(figsize=(9, 9))
ax = plt.axes()

ax.hist(source_catalog["z"], bins=50, alpha=0.3, density=True, label="discrete values")
ax.axvline(zsrc_min, color="red")
# here we multiply by a constant for visualisation purposes
ax.plot(
    z,
    model_z_distrib_dict["func"](z) * 25,
    linestyle="dashed",
    label="distribution function",
)

ax.text(
    1.75,
    0.9,
    f"Resulting efficiency\n${beta_label(beta_s_mean)}$\n${beta_sq_label(beta_s_square_mean)}$",
    bbox={"boxstyle": "round", "alpha": 0.5, "edgecolor": ".7", "facecolor": "1"},
    fontsize=8,
)
ax.text(zsrc_min, 0.03, "$z_{min}$", color="r", fontsize=8, rotation=30)


ax.set_xlim(0, 2.99)
ax.set_xlabel("redshift")
ax.set_ylabel("distribution")
ax.set_yticklabels([])
ax.legend(fontsize=8)

add_grid(ax)
plt.tight_layout()
plt.savefig("theo_diff_z_types.png")

## Profiles

In [ ]:
z_inf = 1000

beta_s_mean = clmm.utils.compute_beta_s_mean_from_distribution(
    cluster_z,
    z_inf,
    cosmo,
    zmax=zsrc_max,
    delta_z_cut=delta_z_cut,
    zmin=None,
    z_distrib_func=model_z_distrib_dict["func"],
)
beta_s_square_mean = clmm.utils.compute_beta_s_square_mean_from_distribution(
    cluster_z,
    z_inf,
    cosmo,
    zmax=zsrc_max,
    delta_z_cut=delta_z_cut,
    zmin=None,
    z_distrib_func=model_z_distrib_dict["func"],
)

display(Math(beta_label(beta_s_mean)))
display(Math(beta_sq_label(beta_s_square_mean)))

In [ ]:
prof_kwargs = dict(
    r_proj=np.logspace(np.log10(0.2), np.log10(5), 10),
    mdelta=cluster_m,
    cdelta=concentration,
    z_cluster=cluster_z,
    cosmo=cosmo,
    delta_mdef=500,
    massdef="critical",
)

### g_t

In [ ]:
%%time
gt = {
    key: clmm.theory.compute_reduced_tangential_shear(**prof_kwargs, **_kwargs)
    for key, _kwargs in (
        (
            "exact",
            dict(
                z_src=model_z_distrib_dict["func"],
                z_src_info="distribution",
                approx=None,
            ),
        ),
        (
            "order1",
            dict(
                z_src=[beta_s_mean, beta_s_square_mean],
                z_src_info="beta",
                approx="order1",
            ),
        ),
        (
            "order2",
            dict(
                z_src=[beta_s_mean, beta_s_square_mean],
                z_src_info="beta",
                approx="order2",
            ),
        ),
    )
}

### mu

In [ ]:
%%time
mu = {
    key: clmm.theory.compute_magnification(**prof_kwargs, **_kwargs)
    for key, _kwargs in (
        (
            "exact",
            dict(
                z_src=model_z_distrib_dict["func"],
                z_src_info="distribution",
                approx=None,
            ),
        ),
        (
            "order1",
            dict(
                z_src=[beta_s_mean, beta_s_square_mean],
                z_src_info="beta",
                approx="order1",
            ),
        ),
        (
            "order2",
            dict(
                z_src=[beta_s_mean, beta_s_square_mean],
                z_src_info="beta",
                approx="order2",
            ),
        ),
    )
}

### delta_mu

In [ ]:
alpha = [2, -0.5]

In [ ]:
%%time
dmu = {
    key: [
        clmm.theory.compute_magnification_bias(**prof_kwargs, **_kwargs, alpha=_alpha)
        for _alpha in alpha
    ]
    for key, _kwargs in (
        (
            "exact",
            dict(
                z_src=model_z_distrib_dict["func"],
                z_src_info="distribution",
                approx=None,
            ),
        ),
        (
            "order1",
            dict(
                z_src=[beta_s_mean, beta_s_square_mean],
                z_src_info="beta",
                approx="order1",
            ),
        ),
        (
            "order2",
            dict(
                z_src=[beta_s_mean, beta_s_square_mean],
                z_src_info="beta",
                approx="order2",
            ),
        ),
    )
}

### Plot

In [ ]:
def plot_cases(axes, radius, profiles):
    axes[0].loglog(radius, profiles[0][0], **profiles[0][-1])
    for case in profiles[1:]:
        axes[0].loglog(radius, case[0], **case[-1])
        axes[1].plot(radius, 100 * (case[0] / profiles[0][0] - 1), **case[-1])

    axes[1].set_xlabel("R [Mpc]")
    for ax in axes:
        add_grid(ax)

In [ ]:
from matplotlib.ticker import NullFormatter

In [ ]:
fig, axes = prep_plot(
    figsize=(18, 7),
    subplots=(2, 3),
    subplots_kwargs=dict(sharex=True, height_ratios=[4, 1]),
)
plot_cases(
    axes[:, 0],
    prof_kwargs["r_proj"],
    [
        (gt["exact"], {"label": "No Approx.", "color": "0", "lw": 0.8}),
        (gt["order1"], {"label": "Order 1", "color": "C0"}),
        (gt["order2"], {"label": "Order 2", "color": "C1"}),
    ],
)
plot_cases(
    axes[:, 1],
    prof_kwargs["r_proj"],
    [
        (mu["exact"], {"label": "No Approx.", "color": "0", "lw": 0.8}),
        (mu["order1"], {"label": "Order 1", "color": "C0"}),
        (mu["order2"], {"label": "Order 2", "color": "C1"}),
    ],
)
plot_cases(
    axes[:, 2],
    prof_kwargs["r_proj"],
    [
        (dmu["exact"][0], {"label": "No Approx.", "color": "0", "lw": 0.8}),
        (dmu["order1"][0], {"label": "Order 1", "color": "C0"}),
        (dmu["order2"][0], {"label": "Order 2", "color": "C1"}),
    ],
)
plot_cases(
    axes[:, 2],
    prof_kwargs["r_proj"],
    [
        (dmu["exact"][1], {"label": "No Approx.", "color": "0", "ls": "--", "lw": 0.6}),
        (dmu["order1"][1], {"label": "Order 1", "color": "C0", "ls": "--", "lw":0.8}),
        (dmu["order2"][1], {"label": "Order 2", "color": "C1", "ls": "--", "lw":0.8}),
    ],
)
axes[0, 0].legend(fontsize=8)
axes[1, 0].set_ylabel("%")

for ax, label in zip(axes[0], ("$\gamma_t$", "$\mu_t$", "$\delta_{\mu_t}$")):
    ax.set_title(label, fontsize=10)

for ax in axes.flatten():
    ax.tick_params(axis="both", which="major", labelsize=8)
    #ax.tick_params(axis="both", which="minor", labelsize=8)
    ax.yaxis.set_minor_formatter(NullFormatter())
    ax.set_xlim(.2, 5)

handles, labels = axes[0, -1].get_legend_handles_labels()
axes[0, -1].legend(
    handles[::3],
    [fr"$\alpha={_alpha}$" for _alpha in alpha],
    #loc=(0.38, 0.8),
    fontsize=8,
)


plt.tight_layout()
plt.subplots_adjust(hspace=0)

plt.savefig("theo_diff_z_types_profiles.png")